In [1]:
import time

In [2]:
from tensorflow.keras.models import load_model

2024-04-09 07:18:09.022091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 07:18:09.588431: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from global_best import GlobalBestPSO

In [4]:
import numpy as np

In [5]:
loaded_model = load_model('surogate.h5')

2024-04-09 07:18:11.716630: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-04-09 07:18:11.717124: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5
2024-04-09 07:18:11,906 - tensorflow - WARNING - No training configuration found in the save file, so the model was *not* compiled. Compile it manually.


In [6]:
# 特定频率
def loss_freq(s_para):
    # 能量
    E = np.square(s_para)
    E11 = E[:,0] + E[:,1]
    E21 = E[:,2] + E[:,3]
    E31 = E[:,4] + E[:,5]
    E41 = E[:,6] + E[:,7]
    P21 = np.arctan2(s_para[:,3], s_para[:,2])
    P31 = np.arctan2(s_para[:,5], s_para[:,4])

    # 压制
    loss1 = E11 - E21 - E31 + E41
    # 比例
    loss2 = np.abs(E21 / (E31 + E21) - 0.7)
    # phase
    loss3 = np.abs(P21 - P31 - np.pi / 4)

    loss4 = np.abs(np.sum(E, axis = -1) - 1)
    
    return loss1 + loss2 + loss3 + loss4

In [7]:
def dgn_obj(s1, s2, s3):
    # return np.max([loss_freq(s1), loss_freq(s2), loss_freq(s3)], axis=0)
    return loss_freq(s1) + loss_freq(s2) + loss_freq(s3)

In [8]:
# 参数数量，迭代次数
EPOCHS = 100

In [9]:
# 定义目标函数
def objective_function(x):
    n_samples = x.shape[0]
    inputs1 = np.concatenate((freq1[:n_samples], x), axis=-1)
    inputs2 = np.concatenate((freq2[:n_samples], x), axis=-1)
    inputs3 = np.concatenate((freq3[:n_samples], x), axis=-1)
    y_pred1 = loaded_model.predict(inputs1, batch_size=n_samples, verbose=0)
    y_pred2 = loaded_model.predict(inputs2, batch_size=n_samples, verbose=0)
    y_pred3 = loaded_model.predict(inputs3, batch_size=n_samples, verbose=0)
    return dgn_obj(y_pred1, y_pred2, y_pred3)

In [10]:
freq1 = np.ones((40000, 1), dtype=np.float32) * 2.4
freq2 = np.ones((40000, 1), dtype=np.float32) * 2.5
freq3 = np.ones((40000, 1), dtype=np.float32) * 2.6

In [11]:
# 物理边界
lb = [1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4]
ub = [5, 5, 5, 5, 5, 5, 5, 5, 100, 100, 100, 100]

In [12]:
def myOpti(num_nodes):
    # 创建优化器实例，设置边界约束以及其他参数
    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
    optimizer = GlobalBestPSO(n_particles=num_nodes, dimensions=len(lb), bounds=(lb, ub), options=options)
    # 运行PSO
    cost, pos, logs = optimizer.optimize(objective_function, iters=EPOCHS)
    return logs[-1]['time'] / EPOCHS

In [13]:
logs = []

In [14]:
for i in range(200, 20001, 200):
    logs.append(myOpti(i))

2024-04-09 07:18:17,202 - global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
global_best: 100%|█████████████████████████████████████|100/100, best_cost=-1.94
2024-04-09 07:18:28,050 - global_best - INFO - Optimization finished | best cost: -1.9401648044586182, best pos: [ 3.66122429  3.48963859  1.89964158  1.83641755  4.10163203  3.43160618
  3.25404793  3.54149716 53.99912976 32.79634863  9.35171385 51.62956157]
2024-04-09 07:18:28,055 - global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
global_best: 100%|█████████████████████████████████████|100/100, best_cost=-2.41
2024-04-09 07:18:38,569 - global_best - INFO - Optimization finished | best cost: -2.4098806381225586, best pos: [ 3.93334236  4.71394382  1.7830547   3.90681648  4.00391287  3.33104337
  3.09994066  3.81615714 52.84566233 61.29863411 15.62876148  9.74787975]
2024-04-09 07:18:38,577 - global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w':

In [15]:
for log in logs:
    print(log)

0.10845674037933349
0.10510191917419434
0.10353712558746338
0.10779772758483887
0.10604621171951294
0.10707834959030152
0.10548717737197875
0.10941834688186645
0.10909511089324951
0.11139492034912109
0.10827756643295289
0.11222068071365357
0.11070066213607788
0.1117986798286438
0.10977232456207275
0.1148177719116211
0.1105169677734375
0.11563996315002441
0.1131321930885315
0.11550440073013306
0.11403801918029785
0.11753982067108154
0.11560122489929199
0.1166633939743042
0.11483397960662842
0.11910122871398926
0.11727131366729736
0.12052913427352906
0.11920921087265014
0.12040616512298584
0.11857698440551757
0.12286383152008057
0.12130656957626343
0.12452217578887939
0.12333446979522705
0.12461835145950317
0.12318068265914917
0.1262692379951477
0.1261291790008545
0.12807440042495727
0.12609845876693726
0.12879709005355836
0.12606517791748048
0.12862316608428956
0.12736990928649902
0.13081043243408202
0.12891000986099244
0.13031693696975707
0.1310216999053955
0.13423853397369384
0.129564

2024-04-09 02:41:55,146 - global_best - INFO - Optimize for 3000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
global_best: 100%|███████████████████████████████████|3000/3000, best_cost=-2.71
2024-04-09 02:47:07,564 - global_best - INFO - Optimization finished | best cost: -2.714648485183716, best pos: [ 3.29515405  3.8248283   1.82010941  2.49914734  3.84600031  4.12669532
  2.18406875  3.59010014 14.30716111 63.98216545 15.60262511 49.70426199]


In [14]:
logs

[{'epoch': 0, 'time': 0.7083697319030762, 'acc': 0.47924309968948364},
 {'epoch': 1, 'time': 0.8049442768096924, 'acc': 0.2509128153324127},
 {'epoch': 2, 'time': 0.9174425601959229, 'acc': 0.2509128153324127},
 {'epoch': 3, 'time': 1.029639720916748, 'acc': 0.2509128153324127},
 {'epoch': 4, 'time': 1.1417572498321533, 'acc': 0.15343111753463745},
 {'epoch': 5, 'time': 1.2385895252227783, 'acc': -0.905592143535614},
 {'epoch': 6, 'time': 1.3451383113861084, 'acc': -1.3051302433013916},
 {'epoch': 7, 'time': 1.4551033973693848, 'acc': -1.3051302433013916},
 {'epoch': 8, 'time': 1.54948091506958, 'acc': -1.7054861783981323},
 {'epoch': 9, 'time': 1.654942274093628, 'acc': -1.7054861783981323},
 {'epoch': 10, 'time': 1.7709460258483887, 'acc': -1.7054861783981323},
 {'epoch': 11, 'time': 1.8851039409637451, 'acc': -1.7054861783981323},
 {'epoch': 12, 'time': 1.9857828617095947, 'acc': -1.7054861783981323},
 {'epoch': 13, 'time': 2.0898005962371826, 'acc': -1.7054861783981323},
 {'epoch':

In [15]:
for log in logs:
    print(log['time'], log['acc'])

0.7083697319030762 0.47924309968948364
0.8049442768096924 0.2509128153324127
0.9174425601959229 0.2509128153324127
1.029639720916748 0.2509128153324127
1.1417572498321533 0.15343111753463745
1.2385895252227783 -0.905592143535614
1.3451383113861084 -1.3051302433013916
1.4551033973693848 -1.3051302433013916
1.54948091506958 -1.7054861783981323
1.654942274093628 -1.7054861783981323
1.7709460258483887 -1.7054861783981323
1.8851039409637451 -1.7054861783981323
1.9857828617095947 -1.7054861783981323
2.0898005962371826 -1.7054861783981323
2.1982781887054443 -1.7054861783981323
2.3074233531951904 -1.7054861783981323
2.406557321548462 -1.7054861783981323
2.502685546875 -1.7054861783981323
2.6043336391448975 -1.8130534887313843
2.6984920501708984 -1.9028441905975342
2.7970757484436035 -1.9028441905975342
2.897916078567505 -2.0360124111175537
3.0066967010498047 -2.0441761016845703
3.1142094135284424 -2.0441761016845703
3.222505807876587 -2.079123020172119
3.3251116275787354 -2.079123020172119
3.4